# Predição de Temperatura com GRU - TinyML

**Projeto:** Sistema de Predição de Temperatura para Microcontroladores

**Objetivo:** Prever a temperatura do sensor AHT20 em 5, 10 e 15 minutos no futuro

**Sensores utilizados:**
- AHT20: Temperatura e Umidade
- BMP280: Temperatura e Pressão

**Modelo:** GRU (Gated Recurrent Unit) compatível com TensorFlow Lite Micro

**Deploy:** Raspberry Pi Pico (RP2040)

---

# 1. Carregar os Dados

---

In [ ]:
# Bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# TensorFlow e Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GRU, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import regularizers

# Sklearn para pré-processamento e métricas
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import os
import joblib

# Configuração de visualização
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Configurar seed para reprodutibilidade
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

print(f'TensorFlow: {tf.__version__}')
print(f'Seed configurado: {SEED}')

In [ ]:
#Carregar o dataset de temperatura em lê csv
df_raw = pd.read_csv('data/temp.csv')
df_raw['Timestamp'] = pd.to_datetime(df_raw['Timestamp'])#converter coluna de timestamp para datetime
print(f'Shape original: {df_raw.shape}')
print(f'Colunas disponíveis: {list(df_raw.columns)}')

In [ ]:
#Selecionar apenas as features de interesse
features = ['Temp_AHT20_C', 'Umid_AHT20_pct', 'Temp_BMP280_C', 'Press_BMP280_hPa']
df = df_raw[['Timestamp'] + features].copy()

print(f'\nDataset filtrado:')
print(f'Shape: {df.shape}')
print(f'Features selecionadas: {features}')
print(f'Período dos dados: {df["Timestamp"].min()} até {df["Timestamp"].max()}')
print(f'\nPrimeiras linhas:')
df.head()

# 2. Pré-processamento dos Dados

---

In [ ]:
# Criar df_clean 
df_clean = df.copy()

#: aplicar remoção de outliers 
for col in features:
    Q1 = df_clean[col].quantile(0.25)
    Q3 = df_clean[col].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]

print(f'Depois: {len(df_clean)} amostras')
removed = len(df) - len(df_clean)
print(f'Removidos: {removed} ({100*removed/len(df):.2f}%)')

## 2.1. Criar Sequências Temporais

**Conceito:** Transformar dados tabulares em sequências 3D para GRU

**Formato:**
- Input (X): Janela de 10 timesteps com 4 features cada
- Output (y): Temperatura futura em 3 horizontes (5, 10, 15 minutos)

**Exemplo:**
```
X = [timestep_0, timestep_1, ..., timestep_9]  <- 10 timesteps passados
y = [temp_t+10, temp_t+19, temp_t+29]           <- 3 previsões futuras
```

In [ ]:
#Configurar parâmetros das sequências
WINDOW_SIZE = 10 #Tamanho janela
HORIZONS = [10, 19, 29] # tempo para cada previsao (5 minutos, 10 minutos, 15 minutos)
print(f'  - Prever temperatura em {HORIZONS[0] * 31 / 60:.1f}, {HORIZONS[1] * 31 / 60:.1f} e {HORIZONS[2] * 31 / 60:.1f} minutos')

In [ ]:
def create_sequences(data, window_size, horizons):
    """
    Criar sequências temporais para treinamento
    
    Parâmetros:
    - data: DataFrame com os dados
    - window_size: tamanho da janela de observação
    - horizons: lista de horizontes de previsão

    Retorna:
    - X: array 3D (samples, timesteps, features) -> Amostras e janelas temporais
    - y: array 2D (samples, horizons) -> TARGET É APENAS OS VALORES DE PREVISAO PARA 5, 10 E 15 MINUTOS
    """
    X, y = [], [] #armazenar x e y
    target = 'Temp_AHT20_C'
    
    for i in range(len(data) - window_size - max(horizons)): #itera pelos dados, parando antes do final para garantir espaço suficiente
        #para a janela + horizonte máximo de previsão
        X.append(data.iloc[i:i+window_size][features].values) #Seleciona apenas as colunas de features 
        #Pega uma janela de tamanho window_size começando na posição i
        y.append([data.iloc[i+window_size+h][target] for h in horizons])#Para cada horizonte em horizons (ex: [0, 5, 10]), pega o valor do target no futuro
        #i+window_size+h = posição após a janela + horizonte
        #Cria uma lista com múltiplos targets (previsões para diferentes tempos futuros)
    
    return np.array(X), np.array(y)

X, y = create_sequences(df_clean, WINDOW_SIZE, HORIZONS) #APLICA FUNÇÃO

print(f'\nX shape: {X.shape}')
print(f'  - {X.shape[0]} amostras')
print(f'  - {X.shape[1]} timesteps por amostra')
print(f'  - {X.shape[2]} features por timestep')
print(f'\ny shape: {y.shape}')
print(f'  - {y.shape[0]} amostras')
print(f'  - {y.shape[1]} horizontes de previsão')

## 2.2. Divisão em Treino, Validação e Teste

**Divisão:**
- Treino: 70% dos dados (para treinar o modelo)
- Validação: 15% dos dados (para ajustar hiperparâmetros)
- Teste: 15% dos dados (para avaliar desempenho final)

In [ ]:
#Dividir dados em treino, validação e teste
train_size = int(0.7 * len(X))
val_size = int(0.85 * len(X))

X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:val_size], y[train_size:val_size]
X_test, y_test = X[val_size:], y[val_size:]

print('Dados divididos:')
print(f'  Treino:    {len(X_train):6d} amostras ({100*len(X_train)/len(X):.1f}%)')
print(f'  Validação: {len(X_val):6d} amostras ({100*len(X_val)/len(X):.1f}%)')
print(f'  Teste:     {len(X_test):6d} amostras ({100*len(X_test)/len(X):.1f}%)')

## 2.3. Normalização dos Dados

**Por que normalizar?**
- Features têm escalas diferentes (temperatura ~20°C, pressão ~920 hPa)
- Redes neurais aprendem melhor com dados normalizados
- Evita que features com valores grandes dominem o treinamento

**Método:** StandardScaler (Z-score normalization)
- Transforma dados para média = 0 e desvio padrão = 1
- Fórmula: (x - média) / desvio_padrão

In [ ]:
scaler = StandardScaler()# Normalizar dados usando StandardScaler

X_train_reshaped = X_train.reshape(-1, X_train.shape[-1]) #Transforma dados 3D (amostras, timesteps, features) em 2D (amostras*timesteps, features)
#Necessário porque StandardScaler trabalha com dados 2D
scaler.fit(X_train_reshaped) #Calcula média e desvio padrão de cada feature usando APENAS dados de treino
#Aprende os parâmetros de normalização

#aplica (x - média) / desvio para cada feature
# Processo: reshape para 2D → normaliza → reshape de volta para 3D
#Os MESMOS parâmetros aprendidos no treino são aplicados em val e test
X_train_scaled = scaler.transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_val_scaled = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)
X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
print(f'\nParâmetros do scaler:')
print(f'  Média (treino): {X_train_scaled.mean():.6f}')
print(f'  Desvio (treino): {X_train_scaled.std():.6f}')
print(f'\nMédias por feature:')
for i, name in enumerate(features):
    print(f'  {name:20s}: mean={scaler.mean_[i]:8.4f}, std={scaler.scale_[i]:8.4f}')

# 3. Construção e Treinamento do Modelo

---

## 3.1. Arquitetura do Modelo GRU

**Por que GRU?**
- Rede recorrente projetada para capturar dependências temporais de longo prazo
- Mais leve que LSTM (2 gates vs 3 gates), ideal para TinyML
- Processa a sequência timestep a timestep mantendo estado oculto (memória)
- Compatível com TensorFlow Lite Micro

**Arquitetura:**
```
Input (10 timesteps × 4 features)
    |
GRU(24 unidades) + Dropout(0.2)
    |
Dense(16) + ReLU + Dropout(0.2)
    |
Dense(3) [Output: 3 previsões]
```

In [ ]:
# Construir modelo GRU
model = Sequential([
    # Camada de entrada
    Input(shape=(WINDOW_SIZE, 4)),  #Entrada: janela temporal com 4 features
    
    # Camada GRU - processa a sequência temporal mantendo estado oculto
    # return_sequences=False: retorna apenas o último estado oculto (vetor de 24 valores)
    GRU(24,
        kernel_regularizer=regularizers.l2(0.0001),#Regularização L2 para evitar overfitting
        recurrent_regularizer=regularizers.l2(0.0001),#Regularização nos pesos recorrentes
        name='gru_1'),
    
    #Dropout para reduzir overfitting (desliga 20% dos neurônios)
    Dropout(0.2, name='dropout_1'),
    
    # Camada densa para combinar features extraídas pela GRU
    Dense(16, activation='relu',
          kernel_regularizer=regularizers.l2(0.0001),
          name='dense_1'),
    
    #Segundo dropout
    Dropout(0.2, name='dropout_2'),
    
    #Camada de saída - 3 valores (previsões para 3 horizontes temporais)
    Dense(3, activation='linear', name='output')
], name='GRU_Temperature')

# Compilar modelo
model.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='mse',
    metrics=['mae']
)

print('\nArquitetura:')
model.summary()

print(f'\nTotal de parâmetros: {model.count_params():,}')

## 3.2. Configurar Callbacks

**Callbacks** são funções que executam durante o treinamento:

1. **EarlyStopping**: Para o treinamento se a validação não melhorar
2. **ReduceLROnPlateau**: Reduz learning rate quando estagnar

In [ ]:
# Configurar callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=50, #aguarda 50 epocas sem melhora no val_loss antes de parar
        restore_best_weights=True, #restaura pesos da melhor epoca
        verbose=1#mostra quando para
    ), 
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5, #reduz learning rate pela metade
        patience=20, #aguarda 20 epoca sem melhorar antes de reduzir
        min_lr=1e-7, #minimo
        verbose=1 #mostrar quando reduzir
    )
]

## 3.3. Treinar o Modelo

In [ ]:
history = model.fit( #treinamento modelo
    X_train_scaled, y_train, # x e y de treino sendo x o normalizado
    validation_data=(X_val_scaled, y_val), #dados de validação
    epochs=300, #300 epoca
    batch_size=512,#Pega 512 amostras do dataset, gradiente mais estavel
    callbacks=callbacks, #Callbacks mostrado acima
    verbose=1 #mostra andamento
)

## 3.4. Visualizar Curvas de Aprendizado

In [ ]:
# Criar pasta images/GRU se não existir
os.makedirs('images/GRU', exist_ok=True)

# Plotar curvas de loss e MAE
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

axes[0].plot(history.history['loss'], label='Treino', linewidth=2)
axes[0].plot(history.history['val_loss'], label='Validação', linewidth=2)
axes[0].set_xlabel('Época', fontsize=12)
axes[0].set_ylabel('Loss (MSE)', fontsize=12)
axes[0].set_title('Curva de Loss', fontweight='bold', fontsize=14)
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

axes[1].plot(history.history['mae'], label='Treino', linewidth=2)
axes[1].plot(history.history['val_mae'], label='Validação', linewidth=2)
axes[1].set_xlabel('Época', fontsize=12)
axes[1].set_ylabel('MAE (°C)', fontsize=12)
axes[1].set_title('Curva de MAE', fontweight='bold', fontsize=14)
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('images/GRU/01_curvas_aprendizado.png', dpi=300, bbox_inches='tight')
plt.show()

# 4. Avaliação do Modelo

---

## 4.1. Métricas no Conjunto de Teste

**Métricas utilizadas:**
- **MAE** (Mean Absolute Error): Erro médio absoluto em °C
- **RMSE** (Root Mean Squared Error): Raiz do erro quadrático médio
- **R²** (Coefficient of Determination): Quanto o modelo explica a variância

In [ ]:
y_pred = model.predict(X_test_scaled, verbose=0)#Fazer predições no conjunto de teste
# Calcular métricas gerais no conjunto de teste
mae_overall = mean_absolute_error(y_test, y_pred)
rmse_overall = np.sqrt(mean_squared_error(y_test, y_pred))
r2_overall = r2_score(y_test, y_pred)

# Calcular MAE por horizonte
mae_t1 = mean_absolute_error(y_test[:, 0], y_pred[:, 0]) #5 minutos
mae_t2 = mean_absolute_error(y_test[:, 1], y_pred[:, 1])#10 minutos
mae_t3 = mean_absolute_error(y_test[:, 2], y_pred[:, 2]) #15 minutos

print(f'\nMétricas Gerais:')
print(f'  MAE:  {mae_overall:.4f} °C')
print(f'  RMSE: {rmse_overall:.4f} °C')
print(f'  R²:   {r2_overall:.4f}')
print(f'\nMAE por Horizonte:')
print(f'  T+10 (5 min):   {mae_t1:.4f} °C')
print(f'  T+19 (10 min):  {mae_t2:.4f} °C')
print(f'  T+29 (15 min):  {mae_t3:.4f} °C')
print(f'\nInformações do Modelo:')
print(f'  Parâmetros totais: {model.count_params():,}')

## 4.2. Gráficos de Predito vs Real

In [ ]:
# Plotar scatter plots para cada horizonte
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

horizons_names = ['T+10 (5 min)', 'T+19 (10 min)', 'T+29 (15 min)']

for i, (horizon_name, ax) in enumerate(zip(horizons_names, axes)):
    ax.scatter(y_test[:, i], y_pred[:, i], alpha=0.4, s=15)
    min_val = min(y_test[:, i].min(), y_pred[:, i].min())
    max_val = max(y_test[:, i].max(), y_pred[:, i].max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Ideal')
    ax.set_xlabel('Real (°C)', fontsize=12)
    ax.set_ylabel('Predito (°C)', fontsize=12)
    ax.set_title(horizon_name, fontweight='bold', fontsize=14)
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)
    mae = mean_absolute_error(y_test[:, i], y_pred[:, i])
    r2 = r2_score(y_test[:, i], y_pred[:, i])
    ax.text(0.05, 0.95, f'MAE: {mae:.3f}°C\nR²: {r2:.3f}',
            transform=ax.transAxes, va='top', fontsize=11,
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7))

plt.tight_layout()
plt.savefig('images/GRU/02_scatter_predictions.png', dpi=300, bbox_inches='tight')
plt.show()

print('Gráfico salvo: images/GRU/02_scatter_predictions.png')

## 4.3. Séries Temporais de Predições

In [ ]:
# Plotar série temporal das predições
n_samples = 300
fig, axes = plt.subplots(3, 1, figsize=(16, 12))

for i, (horizon_name, ax) in enumerate(zip(horizons_names, axes)):
    ax.plot(y_test[:n_samples, i], label='Real', linewidth=2, alpha=0.8)
    ax.plot(y_pred[:n_samples, i], label='Predito', linewidth=2, alpha=0.8)
    ax.set_ylabel('Temperatura (°C)', fontsize=12)
    ax.set_title(horizon_name, fontweight='bold', fontsize=14)
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)

axes[2].set_xlabel('Amostra', fontsize=12)
plt.tight_layout()
plt.savefig('images/GRU/03_timeseries_predictions.png', dpi=300, bbox_inches='tight')
plt.show()

print('Gráfico salvo: images/GRU/03_timeseries_predictions.png')

## 4.4. Análise de Erros

In [ ]:
# Calcular erros
errors = y_pred - y_test

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, (horizon_name, ax) in enumerate(zip(horizons_names, axes)):
    ax.hist(errors[:, i], bins=50, alpha=0.7, edgecolor='black')
    ax.axvline(0, color='red', linestyle='--', linewidth=2, label='Erro zero')
    ax.set_xlabel('Erro (°C)', fontsize=12)
    ax.set_ylabel('Frequência', fontsize=12)
    ax.set_title(f'{horizon_name}: Distribuição dos Erros', fontweight='bold', fontsize=14)
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)
    mean_err = errors[:, i].mean()
    std_err = errors[:, i].std()
    ax.text(0.05, 0.95, f'Média: {mean_err:.3f}\nStd: {std_err:.3f}',
            transform=ax.transAxes, va='top', fontsize=11,
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.savefig('images/GRU/04_error_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print('Gráfico salvo: images/GRU/04_error_distribution.png')

# 5. Conversão para TensorFlow Lite

---

## 5.1. Salvar Modelo Original

In [ ]:
# Criar pasta models/GRU se não existir
os.makedirs('models/GRU', exist_ok=True)

model.save('models/GRU/temperature_model.keras')
print('Modelo Keras salvo: models/GRU/temperature_model.keras')

## 5.2. Converter para TensorFlow Lite

In [ ]:
# Converter para TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open('models/GRU/temperature_model.tflite', 'wb') as f:
    f.write(tflite_model)

size_kb = len(tflite_model) / 1024
print(f'Modelo TFLite salvo: models/GRU/temperature_model.tflite')
print(f'Tamanho: {size_kb:.2f} KB')

if size_kb < 100:
    print('Compatível com Raspberry Pi Pico (2MB Flash)')
else:
    print('AVISO: Modelo grande para microcontrolador')

## 5.3. Verificar Conversão TFLite

In [ ]:
# Testar modelo TFLite
interpreter = tf.lite.Interpreter(model_path='models/GRU/temperature_model.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print('Informações do modelo TFLite:')
print(f'  Input shape:  {input_details[0]["shape"]}')
print(f'  Input dtype:  {input_details[0]["dtype"]}')
print(f'  Output shape: {output_details[0]["shape"]}')
print(f'  Output dtype: {output_details[0]["dtype"]}')

test_sample = X_test_scaled[0:1].astype(np.float32)
interpreter.set_tensor(input_details[0]['index'], test_sample)
interpreter.invoke()
tflite_pred = interpreter.get_tensor(output_details[0]['index'])

keras_pred = model.predict(test_sample, verbose=0)

print(f'\nTeste de conversão:')
print(f'  Predição Keras:  {keras_pred[0]}')
print(f'  Predição TFLite: {tflite_pred[0]}')
print(f'  Diferença máxima: {np.abs(keras_pred - tflite_pred).max():.6f} °C')

if np.abs(keras_pred - tflite_pred).max() < 0.01:
    print('\nConversão TFLite: OK')
else:
    print('\nAVISO: Diferença significativa após conversão')

# 6. Geração de Arquivo C para RP2040

---

## 6.1. Converter Modelo TFLite para C Array

In [ ]:
def convert_to_c_array(tflite_model, var_name='model_data'):#Cria um array de bytes em formato C
    c_array = f"const unsigned char {var_name}[] = {{\n"#array de bytes constante (fica na memória flash)
    for i in range(0, len(tflite_model), 12): # # Percorre de 12 em 12 bytes
        line = "  "
        for j in range(12):## Coloca 12 bytes por linha
            if i + j < len(tflite_model):
                line += f"0x{tflite_model[i+j]:02x}, "## Formato hexadecimal
        c_array += line + "\n"
    c_array += "};\n"
    c_array += f"const unsigned int {var_name}_len = {len(tflite_model)};\n"
    return c_array
#Criação do arquivo .h:
h_content = """// Temperature Prediction Model - TinyML (GRU)
// Auto-generated file - Do not edit manually
// Model trained on AHT20 + BMP280 sensor data

#ifndef TEMPERATURE_MODEL_H
#define TEMPERATURE_MODEL_H

// Model information
#define WINDOW_SIZE 10
#define NUM_FEATURES 4
#define NUM_HORIZONS 3

// Feature names
const char* feature_names[] = {
    "Temp_AHT20_C",
    "Umid_AHT20_pct",
    "Temp_BMP280_C",
    "Press_BMP280_hPa"
};

// Horizon names
const char* horizon_names[] = {
    "5 minutes",
    "10 minutes",
    "15 minutes"
};

"""

h_content += convert_to_c_array(tflite_model, 'temperature_model')#Insere o array de bytes do modelo

h_content += """
#endif // TEMPERATURE_MODEL_H
"""

h_filename = 'models/GRU/temperature_model.h'
with open(h_filename, 'w') as f:
    f.write(h_content)

print(f'Arquivo {h_filename} gerado com sucesso!')
print(f'Tamanho do modelo: {len(tflite_model)} bytes ({len(tflite_model)/1024:.2f} KB)')

## 6.2. Gerar Parâmetros do Scaler para C

In [ ]:
# Gerar arquivo header com parâmetros de normalização
scaler_h_content = """// Scaler parameters for normalization
// Auto-generated file - Do not edit manually

#ifndef SCALER_PARAMS_H
#define SCALER_PARAMS_H

// Mean values
const float scaler_mean[] = {
"""

for i, mean_val in enumerate(scaler.mean_):
    scaler_h_content += f"    {mean_val:.6f}f"
    if i < len(scaler.mean_) - 1:
        scaler_h_content += ","
    scaler_h_content += f"  // {features[i]}\n"

scaler_h_content += """};\n
// Scale values
const float scaler_scale[] = {
"""

for i, scale_val in enumerate(scaler.scale_):
    scaler_h_content += f"    {scale_val:.6f}f"
    if i < len(scaler.scale_) - 1:
        scaler_h_content += ","
    scaler_h_content += f"  // {features[i]}\n"

scaler_h_content += """};\n
#endif // SCALER_PARAMS_H
"""

scaler_filename = 'models/GRU/scaler_params.h'
with open(scaler_filename, 'w') as f:
    f.write(scaler_h_content)

print(f'Arquivo {scaler_filename} gerado com sucesso!')
print(f'\nParâmetros do Scaler:')
print(f'  Mean: {scaler.mean_}')
print(f'  Scale: {scaler.scale_}')

## 6.3. Salvar Scaler

In [ ]:
joblib.dump(scaler, 'models/GRU/scaler.pkl')
print('Scaler salvo: models/GRU/scaler.pkl')

# 7. Resumo Final

---

In [ ]:
print('='*70)
print('RESUMO DO PROJETO - GRU')
print('='*70)

print(f'\nDataset:')
print(f'  - Total amostras: {len(df_clean):,}')
print(f'  - Features: {features}')
print(f'  - Target: Temp_AHT20_C')
print(f'  - Período: {df_clean["Timestamp"].min()} até {df_clean["Timestamp"].max()}')

print(f'\nModelo:')
print(f'  - Arquitetura: GRU (1 camada GRU + Dense)')
print(f'  - Parâmetros: {model.count_params():,}')
print(f'  - Tamanho TFLite: {len(tflite_model)/1024:.2f} KB')
print(f'  - Compatível: TensorFlow Lite Micro')

print(f'\nPerformance (Teste):')
print(f'  - MAE Geral:    {mae_overall:.4f} °C')
print(f'  - RMSE:         {rmse_overall:.4f} °C')
print(f'  - R²:           {r2_overall:.4f}')
print(f'  - MAE (5 min):  {mae_t1:.4f} °C')
print(f'  - MAE (10 min): {mae_t2:.4f} °C')
print(f'  - MAE (15 min): {mae_t3:.4f} °C')

print(f'\nArquivos Gerados:')
print(f'  - models/GRU/temperature_model.keras')
print(f'  - models/GRU/temperature_model.tflite')
print(f'  - models/GRU/temperature_model.h')
print(f'  - models/GRU/scaler_params.h')
print(f'  - models/GRU/scaler.pkl')
print(f'  - images/GRU/01-04.png')

print('\n' + '='*70)
print('PROJETO GRU CONCLUÍDO COM SUCESSO!')
print('='*70)